In [23]:
import pandas as pd
import sqlite3
from selenium import webdriver
import math

In [29]:
#testing
driver = webdriver.Chrome(executable_path="c:/chromedriver.exe")
driver.get("http://www.txhighereddata.org/Interactive/CBMStatus")

In [3]:
#element = driver.find_element_by_name('RepCyc')

In [66]:
def convert_date_sqilte(date):
    d,m,y = date.split('/',3)
    return y+'-'+m+'-'+d

reporting_cycle = ['Spring','Fall/Annual','Summer/Summer I']
CBMS = ['CBM001','CBM004','CBM008','CBM00S','CBM009','CBM0E1','CBM00B','FAD001']
years = ['2015','2016','2017']

In [60]:
driver = webdriver.Chrome(executable_path="c:/chromedriver.exe")
new_df = None
issues = []
c = []
d = []
s = []
n = []
conn = sqlite3.connect("data.db")
cur = conn.cursor()
for rc in reporting_cycle:
    #query = "//select[@name='RepCyc']/option[text()='"+ rc+ "']"
    #element=driver.find_element_by_xpath(query).click()
    for year in years:
        #query = "//select[@name='RepYr']/option[text()='"+year+"']"
        #element=driver.find_element_by_xpath(query).click()
        for cbm in CBMS:
            driver.get("http://www.txhighereddata.org/Interactive/CBMStatus")
            query = "//select[@name='RepCyc']/option[text()='"+ rc+ "']"
            element=driver.find_element_by_xpath(query).click()
            
            query = "//select[@name='RepYr']/option[text()='"+year+"']"
            element=driver.find_element_by_xpath(query).click()
            
            element=driver.find_element_by_name("CBMRep")
            query = "//input[@value='"+cbm+"']"
            element = driver.find_element_by_xpath(query).click()

            driver.find_element_by_xpath("//input[@value='Get Status']").click()
            element = driver.find_element_by_class_name("divprime")
            element.get_attribute("id")
            page = element.get_attribute('innerHTML')
            table = pd.read_html(page)
            df =table[2][2:]
            if len(df.columns) != 3:
                st = 'reporting cycle: '+rc+'CBM: '+cbm,'year: '+year
                issues.append(st)
                break
            df.columns = ['College','Status','Date']
            df = df[df['College'].str.contains('The University of Texas')]
            df = df.reset_index(drop=True)



            for index, row in df.iterrows():
                st = row['College']
                #for sqlite you need to have the mm-dd-yyy format
                #print('MISSING DATE ','reporting cycle: ',rc,'CBM: ',cbm,'year: ',year)
                #print('Date is: ',row['Date'] ,"and the type is ",type(row['Date']))
                #date = convert_date_sqilte(row['Date'])
                if type(row['Date']) == float:
                    break
                date = convert_date_sqilte(row['Date']) 
                #TEST CASE
                #if index == 3:
                #    date = '05/07/2008'

                # if the new date is not found in the database
                sql = 'select * from status_table where College= ? and Date= ?'

                temp = pd.read_sql_query(sql,conn,params=(st,date,))
                tqsql = 'Select * from status_table s1 where \
                                  Date=(select Max(Date) from status_table s2 where s1.College=s2.College) \
                                  and College = ?'
                tq = pd.read_sql_query(tqsql,conn,params=(st,))
                #print(temp)
                #temp = pd.read_sql_query('select * from status_table where College="The University of Texas at Austin"',conn)
                # if the record is not there insert it and prepare the new_df to be send out as a notification
                if temp.empty:
                    #print(tq['Date'].values[0])
                    #print(tq['Date'].strftime('%Y-%d-%m'))
                    values = [row['College'],row['Status'],convert_date_sqilte(row['Date'])]
                    cur.execute("insert into status_table values (?, ?, ?)", values)
                    print('Data: ',row['College'],row['Date'],row['Status'])
                    c.append(row['College'])
                    s.append(row['Status'])
                    d.append(row['Date'])
                    n.append(tq['Date'].values[0])
                    
dic = {'College':c,'Status':s,'Old Date':d,'New Date': n}    

new_df = pd.DataFrame.from_dict(dic)
conn.commit()
cur.close()
conn.close()
driver.close()
print(issues)
new_df.head()

Data:  The University of Texas-Rio Grande Valley 04/06/2015 CERTIFIED
Data:  The University of Texas of the Permian Basin 08/06/2015 CERTIFIED
Data:  The University of Texas-Rio Grande Valley 08/06/2015 CERTIFIED
Data:  The University of Texas at El Paso 11/05/2015 CERTIFIED
[('reporting cycle: SpringCBM: CBM009', 'year: 2015'), ('reporting cycle: SpringCBM: CBM009', 'year: 2016'), ('reporting cycle: SpringCBM: CBM009', 'year: 2017'), ('reporting cycle: Summer/Summer ICBM: CBM009', 'year: 2015'), ('reporting cycle: Summer/Summer ICBM: CBM009', 'year: 2016'), ('reporting cycle: Summer/Summer ICBM: CBM009', 'year: 2017')]


,College,Status,Old Date,New Date
0,The University of Texas-Rio Grande Valley,CERTIFIED,04/06/2015,2018-27-03
1,The University of Texas of the Permian Basin,CERTIFIED,08/06/2015,2018-23-07
2,The University of Texas-Rio Grande Valley,CERTIFIED,08/06/2015,2018-27-03
3,The University of Texas at El Paso,CERTIFIED,11/05/2015,2018-29-03


In [46]:
#prepare the new_df for emailing

new_df.columns= ['College','Status','Old Date']

In [61]:
new_df.head()

,College,Status,Old Date,New Date
0,The University of Texas-Rio Grande Valley,CERTIFIED,04/06/2015,2018-27-03
1,The University of Texas of the Permian Basin,CERTIFIED,08/06/2015,2018-23-07
2,The University of Texas-Rio Grande Valley,CERTIFIED,08/06/2015,2018-27-03
3,The University of Texas at El Paso,CERTIFIED,11/05/2015,2018-29-03


In [62]:
html = new_df.to_html()

In [63]:
html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>College</th>\n      <th>Status</th>\n      <th>Old Date</th>\n      <th>New Date</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>The University of Texas-Rio Grande Valley</td>\n      <td>CERTIFIED</td>\n      <td>04/06/2015</td>\n      <td>2018-27-03</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>The University of Texas of the Permian Basin</td>\n      <td>CERTIFIED</td>\n      <td>08/06/2015</td>\n      <td>2018-23-07</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>The University of Texas-Rio Grande Valley</td>\n      <td>CERTIFIED</td>\n      <td>08/06/2015</td>\n      <td>2018-27-03</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>The University of Texas at El Paso</td>\n      <td>CERTIFIED</td>\n      <td>11/05/2015</td>\n      <td>2018-29-03</td>\n    </tr>\n  </tbody>\n</table>'

In [188]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [189]:
me = "my@email.com"
you = "your@email.com"

# Create message container - the correct MIME type is multipart/alternative.
msg = MIMEMultipart('alternative')
msg['Subject'] = "Link"
msg['From'] = me
msg['To'] = you


In [64]:
html = '<html><body>'+html+'</body></html>'

In [65]:
body = = MIMEText(html, 'html')

'<html><body><table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>College</th>\n      <th>Status</th>\n      <th>Old Date</th>\n      <th>New Date</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>The University of Texas-Rio Grande Valley</td>\n      <td>CERTIFIED</td>\n      <td>04/06/2015</td>\n      <td>2018-27-03</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>The University of Texas of the Permian Basin</td>\n      <td>CERTIFIED</td>\n      <td>08/06/2015</td>\n      <td>2018-23-07</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>The University of Texas-Rio Grande Valley</td>\n      <td>CERTIFIED</td>\n      <td>08/06/2015</td>\n      <td>2018-27-03</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>The University of Texas at El Paso</td>\n      <td>CERTIFIED</td>\n      <td>11/05/2015</td>\n      <td>2018-29-03</td>\n    </tr>\n  </tbody>\n</table></body></html>'

In [ ]:
try:
    msg = MIMEText(content, text_subtype)
    msg['Subject']=       subject
    msg['From']   = sender # some SMTP servers will do this automatically, not all

    conn = SMTP(SMTPserver)
    conn.set_debuglevel(False)
    conn.login(USERNAME, PASSWORD)
    try:
        conn.sendmail(sender, destination, msg.as_string())
    finally:
        conn.quit()

except:
    sys.exit( "mail failed; %s" % "CUSTOM_ERROR" ) # give an error message

,0,1,2
2,Angelo State University,CERTIFIED,02/07/2018
3,Lamar University,CERTIFIED,04/25/2018
4,Midwestern State University,CERTIFIED,03/09/2018
5,Prairie View A&M University,CERTIFIED,02/02/2018
6,Sam Houston State University,CERTIFIED,01/31/2018


In [196]:
df.columns = ['College','Status','Date']

In [197]:
df.head()

,College,Status,Date
2,Angelo State University,CERTIFIED,02/07/2018
3,Lamar University,CERTIFIED,04/25/2018
4,Midwestern State University,CERTIFIED,03/09/2018
5,Prairie View A&M University,CERTIFIED,02/02/2018
6,Sam Houston State University,CERTIFIED,01/31/2018


In [198]:
df = df[df['College'].str.contains('The University of Texas')]

In [199]:
df = df.reset_index(drop=True)

In [200]:
df.head()

,College,Status,Date
0,The University of Texas at Arlington,CERTIFIED,07/30/2018
1,The University of Texas at Austin,CERTIFIED,03/19/2018
2,The University of Texas at Dallas,CERTIFIED,06/08/2018
3,The University of Texas at El Paso,CERTIFIED,03/13/2018
4,The University of Texas at San Antonio,CERTIFIED,02/06/2018


In [201]:
#Utility to convert to sqllite date
def convert_date_sqilte(date):
    d,m,y = date.split('/',3)
    return y+'-'+m+'-'+d

In [202]:
# this ned to be exeute only once


#conn = sqlite3.connect("data.db")
#cur = conn.cursor()
#cur.execute("""CREATE TABLE status_table 
#        (College TEXT, Status TEXT, Date TEXT )""")
#cur.close()
#conn.close()

## TODO Fix the dates from mm/dd/yyyy format to yyyy-mm-dd

Series([], Name: College, dtype: object)
Series([], Name: College, dtype: object)
Series([], Name: College, dtype: object)
Series([], Name: College, dtype: object)
Series([], Name: College, dtype: object)
Series([], Name: College, dtype: object)
Series([], Name: College, dtype: object)
Series([], Name: College, dtype: object)


,0,1,2,3,4,5,6,7
College,The University of Texas at Arlington,The University of Texas at Austin,The University of Texas at Dallas,The University of Texas at El Paso,The University of Texas at San Antonio,The University of Texas at Tyler,The University of Texas of the Permian Basin,The University of Texas-Rio Grande Valley
Status,CERTIFIED,CERTIFIED,CERTIFIED,CERTIFIED,CERTIFIED,CERTIFIED,CERTIFIED,CERTIFIED
Date,07/30/2018,03/19/2018,06/08/2018,03/13/2018,02/06/2018,03/14/2018,07/23/2018,04/12/2018


In [27]:
conn = sqlite3.connect("data.db")
cur = conn.cursor()
pd.read_sql_query('Select * from status_table s1 where Date=(select Max(Date) from status_table s2 where s1.College=s2.College)',conn)


,College,Status,Date
0,The University of Texas at Arlington,CERTIFIED,2018-30-07
1,The University of Texas at Tyler,CERTIFIED,2018-14-03
2,The University of Texas of the Permian Basin,CERTIFIED,2018-23-07
3,The University of Texas at Dallas,CERTIFIED,2018-30-01
4,The University of Texas at Austin,CERTIFIED,2018-27-03
5,The University of Texas at El Paso,CERTIFIED,2018-29-03
6,The University of Texas at San Antonio,CERTIFIED,2018-20-03
7,The University of Texas-Rio Grande Valley,CERTIFIED,2018-27-03
